In [26]:
from scipy.signal import hilbert
from scipy.signal import welch
from scipy.signal import butter, lfilter,firls
from scipy import signal
#from utils import METHODS_chaos
import pandas as pd
from scipy import stats
from mne.time_frequency import psd_array_multitaper, psd_array_welch
from fooof import FOOOF
import numpy as np
import argparse
import mne.io
import mne
import os
import multiprocessing as mp
from scipy.io import loadmat
from scipy.io import savemat
import glob
from joblib import Parallel, delayed

import scipy
import sklearn

from tqdm import tqdm



mne.set_log_level("ERROR")





"""
Edge-of-chaos measures
"""

def chaos_pipeline(data, sigma=0.5, denoise=False, downsample='minmax'):
    """Simplified pipeline for the modified 0-1 chaos test emulating the
    implementation from Toker et al. (2022, PNAS). This test assumes
    that the signal is stationarity and deterministic.

    Parameters
    ----------
    data : 1d array
        The (filtered) signal.
    sigma : float, optional
        Parameter controlling the level of noise used to suppress correlations.
        The default is 0.5.
    denoise : bool
        If True, denoising will be applied according to the method by Schreiber
        (2000).
    downsample : str or bool
        If 'minmax', signal will be downsampled by conserving only local minima
        and maxima.

    Returns
    -------
    K: float
        Median K-statistic.

    """
    if denoise:
        # Denoise data using Schreiber denoising algorithm
        data = schreiber_denoise(data)

    if downsample == 'minmax':
        # Downsample data by preserving local minima
        data = _minmaxsig(data)

    # Check if signal is long enough, else return NaN
    if len(data) < 20:
        return np.nan

    # Normalize standard deviation of signal
    x = data * (0.5 / np.std(data)) # matlab equivalent  np.std(data,ddof=1)

    # Mdified 0-1 chaos test
    K = z1_chaos_test(x, sigma=sigma)

    return K


def z1_chaos_test(x, sigma=0.5, rand_seed=0):
    """Modified 0-1 chaos test. For long time series, the resulting K-statistic
    converges to 0 for regular/periodic signals and to 1 for chaotic signals.
    For finite signals, the K-statistic estimates the degree of chaos.

    Parameters
    ----------
    x : 1d array
        The time series.
    sigma : float, optional
        Parameter controlling the level of noise used to suppress correlations.
        The default is 0.5.
    rand_seed : int, optional
        Seed for random number generator. The default is 0.

    Returns
    -------
    median_K : float
        Indicator of chaoticity. 0 is regular/stable, 1 is chaotic and values
        in between estimate the degree of chaoticity.

    References
    ----------
    Gottwald & Melbourne (2004) P Roy Soc A - Math Phy 460(2042), 603-11.
    Gottwald & Melbourne (2009) SIAM J Applied Dyn Sys 8(1), 129-45.
    Toker et al. (2022) PNAS 119(7), e2024455119.
    """
    np.random.seed(rand_seed)
    N = len(x)
    j = np.arange(1,N+1)
    t = np.arange(1,int(round(N / 10))+1)
    M = np.zeros(int(round(N / 10)))
    # Choose a coefficient c within the interval pi/5 to 3pi/5 to avoid
    # resonances. Do this 1000 times.
    c = np.pi / 5 + np.random.random_sample(1000) * 3 * np.pi / 5
    k_corr = np.zeros(1000)

    for its in range(1000):
        # Create a 2-d system driven by the data
        #p = cumsum(x * cos(a * c[i]))
        #q = cumsum(x * sin(a * c[i]))
        p=np.cumsum(x * np.cos(j*c[its]))
        q=np.cumsum(x * np.sin(j*c[its]))

        for n in t:
            # Calculate the (time-averaged) mean-square displacement,
            # subtracting a correction term (Gottwald & Melbourne, 2009)
            # and adding a noise term scaled by sigma (Dawes & Freeland, 2008)

            #M[n-1]=(np.mean((p[n+1:N] - p[1:N-n])**2 + (q[n+1:N]-q[1:N-n])**2)
            #      - np.mean(x)**2 * (1-np.cos(n*c[its])) / (1-np.cos(c[its]))
            #      + sigma * (np.random.random()-.5))

            M[n-1]=(np.mean((p[n:N] - p[:N-n])**2 + (q[n:N]-q[:N-n])**2)
                  - np.mean(x)**2 * (1-np.cos(n*c[its])) / (1-np.cos(c[its]))
                  + sigma * (np.random.random()-.5))

        k_corr[its], _ = scipy.stats.pearsonr(t, M)
        median_k = np.median(k_corr)

    return median_k

def _minmaxsig(x):
    maxs = scipy.signal.argrelextrema(x, np.greater)[0]
    mins = scipy.signal.argrelextrema(x, np.less)[0]
    minmax = np.concatenate((mins, maxs))
    minmax.sort(kind='mergesort')
    return x[minmax]





def fixed_chaos(trial, epochs, lpfrequency):
    K_ch = []
    hfreq = []
    failed = []

    # select trial from epoch
    data_trial = epochs[trial]
    fs = data_trial.shape[1]/10
    nr_channels =  epochs.shape[1]

    for ch in range(nr_channels):
        # select channel data
        data_ch = data_trial[ch,:]
        ch_filt = mne.filter.filter_data(data_ch, sfreq=fs, l_freq=0.5, h_freq=lpfrequency,verbose=False)
        #K_tmp = METHODS_chaos.chaos_pipeline(ch_filt)
        K_tmp = chaos_pipeline(ch_filt)
        K_ch.append(K_tmp)
        hfreq.append(lpfrequency)
        if type(K_tmp) != np.nan:
            failed.append(0)
        else:
            failed.append(1)

    #print(f'Done Trial {str(trial)} Fixed {str(lpfrequency)} Hz')

    return K_ch, hfreq, failed


def filter_and_chaos(trial, epochs):
    K_ch = []
    hfreq = []
    failed = []

    # select trial from epoch
    data_trial = epochs[trial]
    fs = data_trial.shape[1]/10

    nr_channels =  epochs.shape[1]
    for ch in range(nr_channels):
        # select channel data
        data_ch = data_trial[ch,:]
        # do FOOOF to find lowst frequency peak
        fm = FOOOF()
        # Set the frequency range to fit the model
        freq_range = [1, 6]
        # get psd of channels
        freqs, psd_ch = signal.welch(data_ch,fs,nperseg=5*1024)

        fm.fit(freqs, psd_ch, freq_range)

        if fm.peak_params_.shape[0] == 0:
            #no peak found, output nan
            failed.append(1)
            hfreq.append( np.NaN )
            K_ch.append( np.NaN )
        elif fm.peak_params_.shape[0] >= 1:
            failed.append(0)
            # select lowest frequency peak
            peak = fm.peak_params_[np.where(fm.peak_params_[:,0] == np.min(fm.peak_params_[:,0]))[0][0]]
            hfreq_tmp = peak[0] + 0.5*peak[2] #higher edge of lowest frequency
            #Filter data at chosen lowest frequency
            ch_filt = mne.filter.filter_data(data_ch, sfreq=fs, l_freq=0.5, h_freq=hfreq_tmp,verbose=False)
            #K_ch.append(METHODS_chaos.chaos_pipeline(ch_filt))
            K_ch.append(chaos_pipeline(ch_filt))
            hfreq.append(hfreq_tmp)
    
    # print('Done Trial {}'.format(str(trial))) ## Uncomment this if I want to see it. 

    return K_ch, hfreq, failed



In [27]:

out_dir = 'data/output/EOC/'
in_dir = 'data/input/10s/'
k_type = 'fixed'
hfrequ = 4

# output
Freq = []
Nopeak = []
K_median = []
K_space = []
sub_all = []
day_all = []
condition_all = []

'''
if k_type == 'fixed':
    k_type = f"fixed_{str(hfrequ)}"
'''

# make output directory
if not os.path.exists(out_dir):
    os.makedirs(out_dir)

# load patient info and conditions
paths = glob.glob(in_dir + '*.fif')
paths.sort()

#loop over all conditions and particiants
for i, path in tqdm(enumerate(paths)):
    sub = os.path.basename(path)[:4]
    day = os.path.basename(path)[5:9]
    condition = os.path.basename(path)[10:-8]


    #print(f"Analyzing Chaos of {path}");

    #################################
    #          LOAD  DATA          #
    #################################

    #input_fname = f"{in_dir}/epochs_{Drug[i]}_{Cond[i]}_{P_IDS[i]}.mat"
    #data = loadmat(input_fname)
    #epochs = data['trails']

    epochs_mne = mne.read_epochs(path, preload=True)
    epochs_mne.interpolate_bads(reset_bads=True)
    epochs = epochs_mne.get_data()[:,:32,:]

    #epochs = stats.zscore(epochs, axis =1)
    #fs = epochs[0].shape[1]/10
    fs = 256   # CHECK THIS

    #data_test = loadmat('test_data.mat')['data_channel_filt'][0]
    #METHODS_chaos.chaos.chaos_pipeline(data_test)

    # if data is too long only use the first 3 min of data
    nr_trials = min([len(epochs),100]); #adjusted for 100 epochs
    nr_channels =  epochs.shape[1]

    # search individual lowpass frequency
    if k_type == 'indflex':
        fm = FOOOF()
        # Set the frequency range to fit the model
        freq_range = [1, 6]
        data_con = np.concatenate(epochs,axis = 1)
        # get psd of channels
        freqs, psds = signal.welch(data_con,fs,nperseg=5*1024)
        psds =  np.mean(psds,axis = 0)
        fm.fit(freqs, psds, freq_range)
        if fm.peak_params_.shape[0] == 0:
            hfrequ = 4
        if fm.peak_params_.shape[0] >= 1:
            # select lowest frequency peak
            peak = fm.peak_params_[np.where(fm.peak_params_[:,0] == np.min(fm.peak_params_[:,0]))[0][0]]
            hfrequ = peak[0] + 0.5*peak[2] #higher edge of lowest frequency


    #################################
    #    Calculate 01chaos test     #
    #################################

    #pool = mp.Pool(mp.cpu_count())
    # loop over every time segment

    input = []
    for trial in range(nr_trials):
        if k_type == 'indflex':
            input.append((trial,epochs, np.float64(hfrequ)))
        if k_type == 'flex':
            input.append((trial,epochs))
            #filter_and_chaos(trial,epochs) # comment out
        elif k_type == 'fixed':
            input.append((trial,epochs, np.float64(hfrequ)))

        #filter_and_chaos(trial,epochs)
        #fixed_chaos(trial,epochs, np.float(hfrequ))

    #get results for chaos test
    '''
    if k_type == 'flex':
        results = pool.starmap(filter_and_chaos,input)
    else:
        results = pool.starmap(fixed_chaos,input)
    '''
        
    results = Parallel(n_jobs=-1)(delayed(filter_and_chaos)(*args) if k_type == 'flex' else delayed(fixed_chaos)(*args) for args in input)
    

    K_median.append(np.nanmedian(np.array(results)[:,0,:]))
    K_space.append(np.nanmedian(np.array(results)[:,0,:],axis = 0))
    Freq.append(np.nanmean(np.array(results)[:,1,:]))
    Nopeak.append(np.sum(np.array(results)[:,2,:])/(nr_trials*nr_channels))
    sub_all.append(sub)
    day_all.append(day)
    condition_all.append(condition)

    #pool.close()

    # save all lowpass frequencies
    #lp_freqs =np.array(results)[:,1,:]
    #savemat(f"{out_dir}/lpfs/lpf_{Drug[i]}_{Cond[i]}_{P_IDS[i]}.mat", {'lp_freqs': lp_freqs})

    # save part
    output_df = {'sub':sub_all, 'day': day_all,'condition':condition_all,'K_median':K_median,'Freq':Freq,'Nopeak':Nopeak}
    output_df = pd.DataFrame(output_df)
    output_df.to_csv(f'{out_dir}/01Chaos_{k_type}_{hfrequ}.csv', index=False, sep=',')

    # save part
    output_df_space = {'sub': sub_all, 'day': day_all, 'condition': condition_all}
    output_df = pd.concat((pd.DataFrame(output_df_space), pd.DataFrame(K_space).reset_index(drop=True)), axis=1)
    output_df.to_csv(f'{out_dir}/K_space_{k_type}_{hfrequ}.csv', index=False, sep=',')




Analyzing Chaos of data/input/10s/sub0-day1-jhana-epo.fif
Done Trial 4 Fixed 4.0 Hz
Done Trial 6 Fixed 4.0 Hz
Done Trial 2 Fixed 4.0 Hz
Done Trial 3 Fixed 4.0 Hz
Done Trial 11 Fixed 4.0 Hz
Done Trial 1 Fixed 4.0 Hz
Done Trial 10 Fixed 4.0 Hz
Done Trial 0 Fixed 4.0 Hz
Done Trial 8 Fixed 4.0 Hz
Done Trial 5 Fixed 4.0 Hz
Done Trial 9 Fixed 4.0 Hz
Done Trial 7 Fixed 4.0 Hz
Done Trial 12 Fixed 4.0 Hz
Done Trial 14 Fixed 4.0 Hz
Done Trial 13 Fixed 4.0 Hz
Done Trial 17 Fixed 4.0 Hz
Done Trial 20 Fixed 4.0 Hz
Done Trial 22 Fixed 4.0 Hz
Done Trial 21 Fixed 4.0 Hz
Done Trial 15 Fixed 4.0 Hz
Done Trial 16 Fixed 4.0 Hz
Done Trial 18 Fixed 4.0 Hz
Done Trial 19 Fixed 4.0 Hz
Done Trial 23 Fixed 4.0 Hz
Done Trial 24 Fixed 4.0 Hz
Done Trial 25 Fixed 4.0 Hz
Done Trial 26 Fixed 4.0 Hz
Done Trial 28 Fixed 4.0 Hz
Done Trial 27 Fixed 4.0 Hz
Done Trial 29 Fixed 4.0 Hz
Analyzing Chaos of data/input/10s/sub0-day1-mindfulness-epo.fif
Done Trial 4 Fixed 4.0 Hz
Done Trial 2 Fixed 4.0 Hz
Done Trial 3 Fixed 4.0 Hz
